In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
from lib.reproduction import major_oxides
from sklearn.metrics import mean_squared_error
from lib import full_flow_dataloader
import mlflow
import numpy as np
import datetime
import os
os.environ["KERAS_BACKEND"] = "torch"

import torch
import keras


torch.manual_seed(42)
np.random.seed(42)


In [78]:
print(keras.__version__)

3.2.1


In [79]:
import torch.nn as nn
import torch.optim as optim

# Check if GPU is available and set the device accordingly
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


In [80]:
train_processed, test_processed = full_flow_dataloader.load_full_flow_data(load_cache_if_exits=True, average_shots=True)

In [81]:
# cnn_regression_optimized.py
from keras import models, layers, optimizers, regularizers

def build_model(input_dim, output_dim):
    model = models.Sequential()
    model.add(layers.Input(shape=(input_dim,)))
    model.add(layers.Reshape((48, 128, 1)))
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    # Additional convolutional block for better feature extraction
    model.add(layers.Conv2D(32, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(64, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Conv2D(128, (3, 3), activation='relu', padding='same'))
    model.add(layers.BatchNormalization())
    model.add(layers.MaxPooling2D((2, 2)))
    
    model.add(layers.Flatten())
    model.add(layers.Dense(256, activation='relu'))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(output_dim))
    
    # Using L2 regularization
    model.add(layers.Dense(output_dim, kernel_regularizer=regularizers.l2(0.01)))
    
    # Optimizer with a custom learning rate
    optimizer = optimizers.Adam(learning_rate=0.001)
    model.compile(optimizer=optimizer, loss='mse', metrics=['root_mean_squared_error', 'mae'])
    return model

# Constants
INPUT_DIM = 6144  # Number of features per sample
OUTPUT_DIM = 8    # Number of continuous values as output

# Model Creation
model = build_model(INPUT_DIM, OUTPUT_DIM)
model.summary()


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ reshape_10 (Reshape)            │ (None, 48, 128, 1)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_32 (Conv2D)              │ (None, 48, 128, 32)    │           320 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_38          │ (None, 48, 128, 32)    │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_32 (MaxPooling2D) │ (None, 24, 64, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_33 (Conv2D)              │ (None, 24, 64, 32)     │         9,248 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_39          │ (None, 24, 64, 32)     │           128 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_33 (MaxPooling2D) │ (None, 12, 32, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_34 (Conv2D)              │ (None, 12, 32, 64)     │        18,496 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_40          │ (None, 12, 32, 64)     │           256 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_35 (Conv2D)              │ (None, 12, 32, 128)    │        73,856 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ batch_normalization_41          │ (None, 12, 32, 128)    │           512 │
│ (BatchNormalization)            │                        │               │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten_10 (Flatten)            │ (None, 49152)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_22 (Dense)                │ (None, 256)            │    12,583,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout_10 (Dropout)            │ (None, 256)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_23 (Dense)                │ (None, 8)              │         2,056 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_24 (Dense)                │ (None, 8)              │            72 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 12,688,240 (48.40 MB)

 Trainable params: 12,687,728 (48.40 MB)

 Non-trainable params: 512 (2.00 KB)

In [82]:
drop_cols = major_oxides + ["ID", "Sample Name"]

X_train = train_processed.drop(columns=drop_cols)
y_train = train_processed[major_oxides]

X_test = test_processed.drop(columns=drop_cols)
y_test = test_processed[major_oxides]

In [83]:
X_train_reshaped = X_train.to_numpy().reshape(-1, 6144, 1)
X_test_reshaped = X_test.to_numpy().reshape(-1, 6144, 1)

In [84]:
# model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

In [85]:
def run_cnn_experiment(
    X_train: np.ndarray,
    y_train: np.ndarray,
    X_test: np.ndarray,
    y_test: np.ndarray,
    model: keras.Model,
    epochs: int,
    batch_size: int,
    callbacks: list = [],
    major_oxides: list = [],
):
    with mlflow.start_run(run_name="CNN"):
        model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=callbacks)
        y_pred = model.predict(X_test)
        for i, oxide in enumerate(major_oxides):
            y_test_oxide = y_test[:, i]
            y_pred_oxide = y_pred[:, i]
            rmse = mean_squared_error(y_test_oxide, y_pred_oxide, squared=False)
            mlflow.log_metric(f"rmse_{oxide}", float(rmse))

In [86]:
callback = keras.callbacks.EarlyStopping(monitor="val_loss", patience=6, restore_best_weights=True)

class MLFlowCallback(keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs=None):
        if logs is not None:
            for key, value in logs.items():
                mlflow.log_metric(f"{key}", value, step=epoch)


mlflow.set_experiment(f'CNN_Residual_{datetime.datetime.now().strftime("%Y%m%d-%H%M%S")}')
run_cnn_experiment(
    X_train_reshaped,
    y_train.to_numpy(),
    X_test_reshaped,
    y_test.to_numpy(),
    model,
    epochs=1000,
    batch_size=32,
    callbacks=[MLFlowCallback(), callback],
    major_oxides=major_oxides,
)



2024/04/15 12:05:52 INFO mlflow.tracking.fluent: Experiment with name 'CNN_Residual_20240415-120548' does not exist. Creating a new experiment.


Epoch 1/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 3s 57ms/step - loss: 999.4626 - mae: 18.0902 - root_mean_squared_error: 29.8797 - val_loss: 485.8957 - val_mae: 14.3246 - val_root_mean_squared_error: 22.0412
Epoch 2/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 121.0997 - mae: 8.2449 - root_mean_squared_error: 10.9918 - val_loss: 223.6177 - val_mae: 8.3331 - val_root_mean_squared_error: 14.9512
Epoch 3/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 55ms/step - loss: 74.5217 - mae: 6.0492 - root_mean_squared_error: 8.6273 - val_loss: 168.8246 - val_mae: 7.1878 - val_root_mean_squared_error: 12.9902
Epoch 4/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 51ms/step - loss: 66.6037 - mae: 5.6557 - root_mean_squared_error: 8.1555 - val_loss: 125.1990 - val_mae: 6.2623 - val_root_mean_squared_error: 11.1857
Epoch 5/1000
44/44 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - loss: 61.9752 - mae: 5.3776 - root_mean_squared_error: 7.8663 - val_loss: 106.6884 - val_mae: 5.8190 - val_root_mean_squared_error: 10.3252
Epoch 6/1000
44/44 

/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squared' is deprecated in version 1.4 and will be removed in 1.6. To calculate the root mean squared error, use the function'root_mean_squared_error'.
  warnings.warn(
/home/christian/projects/p9/baseline/venv/lib/python3.12/site-packages/sklearn/metrics/_regression.py:483: FutureWarning: 'squa